# Tidy data frames

A **tidy** data frame is one where

- Every row is an observation
- Every column is a feature
- Every cell has a single value

Working with tidy data frames makes downstream analysis much easier.

In [ ]:
import numpy as np
import pandas as pd

## Fix: More than one entry per cell.

In [ ]:
df = pd.DataFrame(dict(a=[1,2,3], b=[(1,2),(2,3),(4,5)], c=['a-1-A', 'b-2-B', 'c-3-C']))
df

In [ ]:
df1 = df.copy()
df1['c'] = df1['c'].str.split('-')
df1

In [ ]:
df2 = df1.explode(column='b').explode(column='c')
df2

In [ ]:
df2.reset_index()

## Fix: Multiple columns for one feature

In [ ]:
df = pd.DataFrame(dict(pid=list('abc'), gene1=[1,2,3], gene2=[4,5,6], gene3=[7,None,None]))
df

In [ ]:
pd.melt(df, id_vars=['pid'])

In [ ]:
df1 = pd.melt(df, 
              id_vars=['pid'], 
              var_name='gene', 
              value_name='expression')
df1

We often use `melt` to make groupby summaries or facet plots.

In [ ]:
(
    df1.
    drop('pid', axis=1).
    groupby(['gene']).
    agg(['count', 'mean'])
)

## Fix: Multiple rows for one observation

Pivot and pivot tabel

In [ ]:
df1

`pivot` behaves like the inverse of `melt`.

In [ ]:
df1.pivot(index='pid', columns='gene', values='expression')

`pivot_table` pivots and aggregates.

In [ ]:
df2 = pd.concat([df1, df1])
df2

In [ ]:
df2.pivot_table(index='pid', values='expression', columns='gene', aggfunc=['mean', 'count'])

## Combining data frames

A tidy data frame is singluar. So we often have to combine them. We have already seen three ways to do so:

- `concat`
- `merge`
- `join`

Run `pip install faker` in terminal first to install package.

In [ ]:
from faker import Faker
fake = Faker()
fake.seed_instance(1234)

#### Combine DataFrames with same strcuture

In [ ]:
cols = ['ssn', 'name', 'email', 'country', 'state', 'county', 'age', 'sex']
data = []
for i in range(10):
    age = np.random.choice(range(18, 100))
    sex = np.random.choice(['M', 'F'])
    person = [fake.ssn(), fake.name(), fake.email(), fake.country(), fake.state(), fake.job(), age, sex]
    data.append(person)
df_employee1 = pd.DataFrame(data, columns=cols)
df_employee1

In [ ]:
cols = ['ssn',  'email', 'name', 'country', 'state', 'county', 'age', 'sex']
data = []
for i in range(10):
    age = np.random.choice(range(18, 100))
    sex = np.random.choice(['M', 'F'])
    person = [fake.ssn(), fake.email(), fake.name(), fake.country(), fake.state(), fake.job(), age, sex]
    data.append(person)
df_employee2 = pd.DataFrame(data, columns=cols)
df_employee2

See what happens to name and email columns.

In [ ]:
df_employees = pd.concat([df_employee1, df_employee2])

In [ ]:
df_employees

You can reset the row labels if you want them to be unique.

In [ ]:
df_employees.reset_index(drop=True)

#### Merging differnt information 

In [ ]:
cols = ['ssn', 'wt', 'ht', 'rhr']
ssns = df_employees.ssn.tolist() + [fake.ssn() for i in range(10)]
n = 10
ssn_sample = np.random.choice(ssns, n, replace=False)
wt = np.random.randint(45, 200, n)
ht = np.random.randint(140, 200, n)
rhr = np.random.randint(40, 120, n)
df_fitness = pd.DataFrame(dict(ssn = ssn_sample, weight = wt, height = ht, hr = rhr))

In [ ]:
df_fitness

When row can be uniquely identified by one or more columns

In [ ]:
df_employees.merge(df_fitness, on='ssn')

In [ ]:
df_employees.merge(df_fitness, on='ssn', how='left')

In [ ]:
df_employees.merge(df_fitness, on='ssn', how='right')

The default is to match on *all* common columns.

In [ ]:
df_employees.merge(df_fitness)

When the dataframes share the same index (row labels)

In [ ]:
df_employees_new = df_employees.set_index('ssn', drop=True)
df_employees_new.head()

In [ ]:
df_fitness_new = df_fitness.set_index('ssn', drop=True)

With the `join` method, you can join multiple DataFrames at the same time. The default is a left join.

In [ ]:
df_employees_new.join([df_fitness_new], how='inner')

In [ ]:
df2.head()

In [ ]:
df2.set_index('pid').head()